In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math
from binarytree import build as btree

As explained in the report, hardcode latencies for critical points and establish a function to obtain the CPU latency between Core #0 and Core#p. Such function trivially interpolates the CPU latency assigning the same value found at the critical point until the next one is reached. 

In [9]:
latencies = {
  "1": 0.14,   # CCX
  "4": 0.35,   # CCD
  "8": 0.35,   # NUMA
  "32": 0.41,  # socket 
  "64": 0.65,  # node 
  "128": 1.82, # cluster
}
def get_latency(p):
  for k in list(latencies)[::-1]:
    if int(k) <= p:
      return latencies[k]
  return latencies[list(latencies.keys())[0]]
df = pd.DataFrame(data={
  "cpu": range(2, 2 + 255),
  "avg_latency": [get_latency(i) for i in range(255)]
}).dropna()

Define a function for computing the execution time for the Binary Tree Broadcast algorithm with np communicating processes. As explained in further detail in the report, the idea is to build a balanced binary tree (using the binarytree Python module) with np nodes, representing the np processes involved in the broadcast. Then, the built tree is traversed recursively from the root keeping track of the maximum cumulative latency (accounted twice) between the traversal of each node's children. This is to appropriately model the parallelized propagation from one layer of the tree to the next one.

In [10]:
def get_bintree_time(np):
  def get_latency(a, b):
    if a // 4 == b // 4:
      return latencies["1"]
    if a // 8 == b // 8:
      return latencies["4"]
    if a // 16 == b // 16:
      return latencies["8"]
    if a // 64 == b // 64:
      return latencies["32"]
    if a // 128 == b // 128:
      return latencies["64"]
    return latencies["128"]
  def traverse(t, total):
    if not t:
      return total
    return max(
      traverse(t.left, total + 2 * get_latency(t.value, t.left.value) if t.left else total),
      traverse(t.right, total + 2 * get_latency(t.value, t.right.value) if t.right else total)
    )
  return traverse(btree(range(0, np)), 0.0)

In [11]:
actual = pd.read_csv("data/20240221_175449_bcast5_msize2.txt")
df["actual_bcast"] = actual
df["estimate_bcast"] = df.apply(lambda e: get_bintree_time(int(e["cpu"])), axis=1)

In [12]:
df

,cpu,avg_latency,actual_bcast,estimate_bcast
0,2,0.14,0.16,0.28
1,3,0.14,0.28,0.28
2,4,0.14,0.18,0.56
3,5,0.14,0.30,0.98
4,6,0.35,0.37,0.98
...,...,...,...,...
250,252,1.82,11.93,8.26
251,253,1.82,11.91,8.26
252,254,1.82,12.41,8.26
253,255,1.82,12.80,8.26


In [13]:
fig = px.scatter(df, x="cpu", y="actual_bcast")
fig.add_trace(go.Scatter(x=df["cpu"], y=df["estimate_bcast"], mode="lines", name="est_bcast"))